# Music_Generation_Via_LSTM's
- By Aman
  - We will be only dealing with the pianos in this project

### Imports

In [2]:
! pip install music21

     |████████████████████████████████| 18.5MB 36.5MB/s eta 0:00:01     |██████████████████████████████  | 17.3MB 36.5MB/s eta 0:00:01
  Stored in directory: /Users/amansaini/Library/Caches/pip/wheels/6e/d0/05/1ef3daa9ae295073d807e468fcd820641965086424f1c633e3
Successfully built music21


In [184]:
import glob
import pickle
import numpy as np                                               # for changing the input shape reshape function of numpy
from keras.utils import np_utils                                 # np_utlis is used for on hot encoding 
from music21 import converter, instrument, note, chord, stream   # Subpackage contain function for preprocessing 

- MIDI (Musical Instrument Digital Interface) is a protocol designed for recording and playing music on digital world
- Musical instruments and other hardware to communicate as maufactures need a simple way to make there products compatible with other brands

## Read a Midi File

In [32]:
# Whenever we have to preprocess the music we deal with midi extension format 
# We dont use mp3 or mp4 format as we dont have libraries for it



# Converter subpacakage has a function known as parse (This is used for reading the midi files)
# .parse is used for reading the midi files

midi = converter.parse("midi_songs/EyesOnMePiano.mid")

In [33]:
midi
# .score is a container present in subpacakage stream which contain both notes and chords 

<music21.stream.Score 0xa269a8990>

In [34]:
# For playing the song show function called by midi object is used with argument as 'midi'
midi.show('midi')

In [35]:
midi.show('text')

# This is the whole song in the text format
# There are some note and there are some chords 
# The number in the front of the text is known as offset (Time at which that note or chord is being played)
# i.e {197.6667} this is offset

{0.0} <music21.stream.Part 0xa262af6d0>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.tempo.MetronomeMark Quarter=95.0>
    {0.0} <music21.key.Key of D major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0xa26642d50>
        {0.0} <music21.note.Note A>
        {0.0} <music21.note.Note A>
        {0.6667} <music21.note.Note G>
        {1.5} <music21.note.Note D>
        {2.0} <music21.note.Note A>
        {2.5} <music21.note.Note G>
        {3.0} <music21.note.Note D>
        {3.5} <music21.note.Note A>
        {4.0} <music21.note.Note A>
        {6.0} <music21.note.Note A>
        {8.0} <music21.note.Note F#>
        {10.0} <music21.chord.Chord F#4 C#5>
        {11.0} <music21.note.Note A>
        {11.6667} <music21.note.Note D>
        {13.3333} <music21.note.Note D>
        {13.6667} <music21.note.Note C#>
        {14.0} <music21.note.Note D>
        {15.0} <music21.note.Note D>
        {16.0} <music21.note.Note G>
        {16.6667} <m

In [37]:
# We have the song in the format of notes and chords 
# but these notes and chords as present in the form of list of list
# i.e [[n,n,c,n,c,c,c,n,],[n,c,c,c,c,c,n,c,c,n,c,n,]____]
# but we want all the notes and chords into a single list hence we will flatten the list
# There are many A in the piano so A3 ,A4 and many more these are called as octave in the piano 
# We can have multiple octaves in the piano depending on the size of the piano



elements_to_parse = midi.flat.notes     # From list of list to a single list

In [38]:
# So now we have 1421 elements to be parsed

len(elements_to_parse)

1421

In [ ]:
for e in elements_to_parse:
    print(e, e.offset)    
    
# There is a special properety of e.offset which shows the offset



# <music21.note.Note F#> 1.0
# <music21.note.Note F#> 1.25
# <music21.note.Note D> 1.5  (It means at time 1.5 we are playing the note D)
# <music21.note.Note D> 1.5  (This time overlapping usually occurs as we have more than one instrument in the song)
# <music21.note.Note C#> 1.75
# <music21.chord.Chord C#> 1.75


# Preprocessing the data 

In [128]:
# fun fact "is instance" return true if the specific object is of specific value say
# x = isinstance("Hello", (float, int, str, list, dict, tuple))




notes_demo = []

for ele in elements_to_parse:
    # If the element is a Note, then store it's pitch
    # note is basically say A but which A say A3 so i.e 3rd batch A is played on the piano 
    # i.e it has a .pitch (takes element as input and return its pitch)
    # .pitch return A3,A4 and so on which stores that which A will be played 
    # converting .pitch to the string 
    # Here note.Note (note is a class with the object of Note)
    if isinstance(ele,note.Note):
        notes_demo.append(str(ele.pitch))
    
    
    # If the element is a Chord, split each note of chord and join them with + say <music21.chord.Chord D4 E4 G4 B4>
    # normalOrder(O is captial) gives the [1,6] say i.e 1 and 6th notes are played
    # i.e normal order returns the list as output
    # So we will convert this into the ['1+6'] say 
    # i.e normal order takes as input <music21.chord.Chord F#4 C#5> and return 4 and 5
    elif isinstance(ele, chord.Chord):
        notes_demo.append("+".join(str(n) for n in ele.normalOrder))

In [131]:
# For notes we have a string like 'A6' and for chords we have like '1+6+4' or '1+3' i.e chord may have any 
# number of element 
len(notes_demo)

1421

In [125]:
isinstance(elements_to_parse[68], chord.Chord)

True

# Preprocessing all Files
- i.e in the above code we have processed only one file (i.e one song) now we have to do this for all songs
- We have to do the same thing for all the files

In [132]:
notes = []


for file in glob.glob("midi_songs/*.mid"):
    
    
    # converter subpackage convert our file to stream.score object 
    # stream.score is nothing just a bucket which contain notes and chords
    midi = converter.parse(file)
    
    print("parsing %s"%file)
    
    
    # Converting list of list to a single list
    elements_to_parse = midi.flat.notes
    
    
    for ele in elements_to_parse:
        if isinstance(ele, note.Note):
            notes.append(str(ele.pitch))

        elif isinstance(ele, chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder))

parsing midi_songs/Ff7-Jenova_Absolute.mid
parsing midi_songs/FF8_Shuffle_or_boogie_pc.mid
parsing midi_songs/ahead_on_our_way_piano.mid
parsing midi_songs/balamb.mid
parsing midi_songs/DOS.mid
parsing midi_songs/FF3_Battle_(Piano).mid
parsing midi_songs/Suteki_Da_Ne_(Piano_Version).mid
parsing midi_songs/caitsith.mid
parsing midi_songs/sera_.mid
parsing midi_songs/Oppressed.mid
parsing midi_songs/ff4-airship.mid
parsing midi_songs/ff7themep.mid
parsing midi_songs/FFVII_BATTLE.mid
parsing midi_songs/electric_de_chocobo.mid
parsing midi_songs/Ff7-One_Winged.mid
parsing midi_songs/Finalfantasy5gilgameshp.mid
parsing midi_songs/Cids.mid
parsing midi_songs/lurk_in_dark.mid
parsing midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
parsing midi_songs/ff4pclov.mid
parsing midi_songs/AT.mid
parsing midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
parsing midi_songs/8.mid
parsing midi_songs/sobf.mid
parsing midi_songs/In_Zanarkand.mid
parsing midi_songs/waltz_de_choco.mid
pars

In [134]:
len(notes) # All Files notes and chords into a single list

60498

In [135]:
# Pickling or marshalling or serilization 
with open("notes", 'wb') as filepath:
    pickle.dump(notes, filepath)

In [136]:
# Unpickling
with open("notes", 'rb') as f:
    notes= pickle.load(f)

- dump is used for pickling and load is used for unpicking 


In [140]:
# Taking total unique elements so taking the set
n_vocab = len(set(notes))

In [141]:
print("Total notes- ", len(notes))
print("Unique notes- ",  n_vocab)

Total notes-  60498
Unique notes-  359


## It means 359 classes are there to feed to our LSTM model and output is also 359 classes

In [144]:
print(notes[100:200])

['G#5', 'F2', 'B-5', 'G5', 'G#5', 'F2', 'B-5', 'C6', 'B-5', 'F2', 'E-6', 'E-2', 'F6', 'E2', '0+5', 'F2', 'F2', 'F2', 'B-2', 'C3', '8+0', 'F2', 'F2', '7+10', 'F2', 'F2', 'C#3', 'C3', '0+3', 'F2', 'F2', 'F2', 'B-2', 'C3', '10+2', 'F2', 'F2', 'F2', 'F2', 'E-2', 'E2', '8+0', 'F2', 'F2', 'F2', 'B-2', 'C3', 'F2', 'F2', 'F2', 'F2', 'C#3', 'C3', '5', 'F2', '5', 'F2', 'B-4', 'F2', 'G4', 'G#4', 'B-2', 'F4', 'G4', 'C3', 'E-4', 'F2', 'F5', 'G5', 'G#5', 'F2', 'B-5', 'G5', 'G#5', 'F2', 'B-5', 'C6', 'B-5', 'F2', 'E-6', 'C#3', 'F6', 'C3', '0+5', 'F2', 'F2', 'F2', 'B-2', 'C3', '8+0', 'F2', 'F2', '7+10', 'F2', 'F2', 'C#3', 'C3', '0+3', 'F2', 'F2']


# Prepare Sequential Data for LSTM

In [168]:
sequence_length = 100

In [159]:
pitchnames = sorted(set(notes))  # All 359 Unique classes from notes

- Deep learning model does not work with categorical data so we have to convert this characters into the numbers (One hot encoding)
- So what we can do is mapping between note to integer

In [165]:
# Saving element and number in the tupule format (we are mapping element with the integer) i.e (ele,num)
# and then calling the dictonary constructor i.e say '0+1+5':1 and 'F2':2 and so on 

ele_to_int = dict((ele, num) for num, ele in enumerate(pitchnames))

In [167]:
network_input = []
network_output = []


- Now listen carefully what we will do here is the same as markov chains 
- i.e we will take the first 0 to 100 element and then 101 is the output
- and then we will slide the window by 1 i.e we will take 1 to 101 element and 102 is the output
- So how many data points we will get is (length of the nodes - sequence length) 
- So we are formulating this as a supervised learning problem i.e taking 100 element and output is 101 element

In [174]:
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length]     # Contain 100 elements
    seq_out = notes[i + sequence_length]      # Gives the 101 th element
    
    
    # But lstm work only with the numeric data and these seq_in and seq_out are characters
    # so we will convert the strings into the integer using the mapping that we have created above
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

    
    
# network_input is a list of list i.e say here each element in the list is further the list of 100 elements
# i.e network_input[0] will print 100 elements 

In [178]:
# No. of examples
n_patterns = len(network_input)
print(n_patterns)

60398


    - lstm takes the data in the 3-D format so we sometimes explictly write 1 i.e ( , ,1) 
    - i.e we use reshape function of numpy to do so
    - for converting into the one hot vector we use the np_utlis of keras specifically to_categorical of keras
    - for changing the input shape reshape function of numpy and to convert to one hot vector utlis function of keras

In [179]:
# lstm  takes the input data into 3_d format so we have to explictly write 1 here i.e (n_patterns, sequence_length, 1)

network_input = np.reshape(network_input,(n_patterns, sequence_length, 1))  # Explicitly wirte one as 3_D data format
print(network_input.shape)

(60398, 100, 1)


-Normalizing the data i.e we have the values in the range 0 to 358 what we can do is we can convert the values between 0 to 1 by dividing with the n_vocab

In [181]:
# Dividing the data with n_vocab (i.e 359) to change the values between 0 and 1
# All the values inside the normalised_network_input are between 0 to 1 after dividing 
# Normalization is important for deep learning as this will converge early and decrease the importance of initial weight



normalised_network_input = network_input/float(n_vocab)

In [188]:
# For one hot vector we use keras np_utlis  (from keras.utlis import np_utlis)
# This np_utlis has a function to_categorical
# np_utils.to_categorical is used to convert array of labeled data(from 0 to nb_classes-1) to one-hot vector.
# Majorily all the ml algorithms need labelled data only some algorithms like decesion trees work well 
# with the categorical data as well 



# Now network output is the one hot vector with 359 cols
# from keras we are importing np_utlis and this further have a function to_categorical
network_output = np_utils.to_categorical(network_output)

In [189]:
network_output.shape
# Shape is changed from 60398 to 359

(60398, 359)

In [190]:
print(normalised_network_input.shape)
print(network_output.shape)


# Now our data is fully prepared to fit into the model
# We have 60398 examples with each example have 100 values and it predicts one output

(60398, 100, 1)
(60398, 359)


# Create Model of LSTM and feed the supervised data that we have created above into the model

- Deep learning models can take hours, days or even weeks to train.
- If the run is stopped unexpectedly, you can lose a lot of work.
- Hence check points are used i.e say ModelCheckPoint 

In [200]:
# For creating the model architecture we are going to use the keras
# This load_model is used when we train the model on google colab and we have to now load the train file on local comp

from keras.layers import *
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

- LSTM tackled the problem of long-term dependencies of RNN in which the RNN cannot predict the word stored in the long term memory but can give more accurate predictions from the recent information

In [195]:
# The simplest model is defined in the Sequential class which is a linear stack of Layers.
# our model is a object of sequential class

model = Sequential()



# First layer of the model is lstm layer
# Since this is the first layer we need to provide the input shape
# return_sequences is true since this is not the last layer of our model 
# return_sequence is refer to as return the hidden state a<t> by default this is set as False
# Here what we passing (100,1) to our model
# 512 is the vector size of hidden layer in LSTM i.e size of activation map
model.add( LSTM(units=512, input_shape = (normalised_network_input.shape[1], normalised_network_input.shape[2]), return_sequences = True))
model.add(Dropout(0.3))




model.add( LSTM(512,return_sequences=True))
model.add( Dropout(0.3) )


model.add( LSTM(512) )# Last layer and by default the return sequence is false


model.add( Dense(256) )# At the end lets adding the dense layer 
model.add( Dropout(0.3) )



# At the end adding the dense layer of 359 i.e (n_vocab) with the softmax (as multiclass classification) 
model.add(Dense(n_vocab, activation="softmax") )

-Adam can be looked at as a combination of RMSprop and Stochastic Gradient Descent with momentum. It uses the squared gradients to scale the learning rate like RMSprop and it takes advantage of momentum by using moving average of the gradient instead of gradient itself like SGD with momentum.

In [197]:
# Compiling the model 
# categorical cross entropy since it has 359 classes


model.compile(loss="categorical_crossentropy", optimizer="adam")

In [198]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
__________

## Train this model on google colab and it takes 8 hours to train on colab

In [201]:
# model.hdf5 this is the filepath in the form of string where the model will be saved 
# loss is the quantity to monitor 
# save_best_only is true then only the best model will be going to be saved based on the quantity we are monitering
# mode tells what is our objective min or max (by default it is minimum for loss and maximum for accuracy)
checkpoint = ModelCheckpoint("model.hdf5", monitor='loss', verbose=0, save_best_only=True, mode='min')



# Same way we can also make the earlystoppig but here we are using only checkpoint








# Training 
# normalised_network_input == Training data  (can be vector,array or matrix)
# network_output           == Traning Labels (can be vector,array or matrix)
# batch_size   == A training dataset can be divided into one or more batches.i.e number of samples before updating 
# the weight as in mini batch gradient descent Popular hyper params for the batch size is 32,64 and 128

model_his = model.fit(normalised_network_input, network_output, epochs=100, batch_size=64, callbacks=[checkpoint])


# We are loading the model that we have generated on google colab
# load_model is used when we want to load the model from colab to local juypter

model = load_model("new_weights.hdf5")

# Predictions

In [204]:
sequence_length = 100
network_input = []



# len(notes) is 60398
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length]  # contains 100 values
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [235]:
# Any random start index
# i.e out of 60398 take any random data point and this data point further have 100 elements in it
# i.e we have 60398 rows each row having 100 elements in it i.e having a list of list
# So we will give these 100 element to the model and model will generate something say it generate 'A3'
# Now we will append 'A3' as 101 element and disgard 1th element


start = np.random.randint(len(network_input) - 1)


# ele_to_int mapping we already have now we are creating the int_to_ele mapping 
int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))





pattern = network_input[start]
prediction_output = []


# generate 200 elements (this is again a hyper parameter like we can generate 10 nodes, 100 nodes or so on here 200)


for note_index in range(200):
    # reshaping the pattern to (1,100,1) ,Last one is because of the 3-D data , First one is the batch size 
    # (means we have 1 data point of 100 length and last 1 is just to convert into the 3_d Data) 
    prediction_input = np.reshape(pattern, (1, len(pattern), 1)) 
    prediction_input = prediction_input/float(n_vocab) # normalizing the prediction 
    
    
    prediction =  model.predict(prediction_input, verbose=0)
    
    
    # Taking the probabilty of the maximum element 
    # Prediction is the vector representing the probabilty of 359 elements
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result) 
    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    # i.e append the output note in the front of list
    # and then discaring the first element i.e moving forward in time 
    pattern.append(idx)
    
    # Take from one till the end 
    pattern = pattern[1:]

In [237]:
print(prediction_output)

['F#3', 'D3', 'F#3', 'B-3', 'C#4', 'B-3', 'F#3', 'C#4', 'B-3', 'F#3', 'D3', 'F#3', 'B3', 'C#4', 'B3', 'F#3', 'C#4', 'B3', 'F#3', 'D3', 'F#3', 'C4', 'C#4', 'C4', 'F#3', 'C#4', 'C4', 'F#3', 'D3', 'F#3', 'B3', 'C#4', 'B3', 'F#3', 'C#4', 'B3', 'F#3', 'D3', 'F#3', 'B-3', 'C#4', 'B-3', 'F#3', 'C#4', 'B-3', 'F#3', 'D3', 'F#3', 'B3', 'C#4', 'B3', 'F#3', 'C#4', 'B3', 'F#3', 'A4', 'D3', 'A3', 'F#3', 'A4', 'A3', 'F#3', 'A4', 'A3', 'F#3', 'C5', 'A3', 'F#3', 'B-4', 'A3', 'F#3', 'A4', 'A3', 'F#3', 'G4', 'G3', 'D4', 'B-3', 'G4', 'D4', 'B-3', 'G4', 'D4', 'B-3', 'G4', 'G2', 'D4', 'B-3', 'G4', 'D4', 'B-3', 'G4', 'D4', 'B-3', 'G4', 'C#3', 'C#4', 'B-3', 'G4', 'C#4', 'B-3', 'G4', 'C#4', 'B-3', 'B-4', 'E3', 'E4', 'C#4', 'A4', 'E4', 'C#4', 'G4', 'E4', 'C#4', 'F4', 'D3', 'D4', 'A3', 'F4', 'D4', 'A3', 'F4', 'D4', 'A3', 'F4', 'D4', 'A3', 'F4', 'D4', 'A3', 'F4', 'D4', 'A3', 'A4', 'D3', 'A3', 'F#3', 'A4', 'A3', 'F#3', 'A4', 'A3', 'F#3', 'C5', 'A3', 'F#3', 'B-4', 'A3', 'F#3', 'A4', 'A3', 'F#3', 'G4', 'D3', 'D4', '

# Create Midi File that can be played from the above 200 predictions

## Till now we have  predicted 200 outputs now we will convert this into the music format 

-Iterating over all the predicted elements and check whether it is a note or chord i.e pattern is note we have to processes it differently if chord then differently 

In [252]:
offset = 0  # Time or we can say position of a particular note or chord in the output
output_notes = []


for pattern in prediction_output:
    
    # if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():   # i.e '1+2+2 ' and '0' both are chord i.e either + or a complete digit
        notes_in_chord = pattern.split('+')     # How many notes in a particular chord 
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))  # Create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        
        new_chord = chord.Chord(temp_notes)         # creates the chord() from the list of notes
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    else:
        # if the pattern is a note
        new_note = note.Note(pattern) # For Converting into the note object i.e B to <music21.note.Note B>
        new_note.offset = offset      # Time at which this particular note is being played
        new_note.storedInstrument = instrument.Piano() # Since This whole data set is of Piano
        output_notes.append(new_note)
        
    offset += 0.5

In [254]:
# create a stream object from the generated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "test_output.mid") # The type is midi and the file name is 'test_output.mid'

'test_output.mid'

In [255]:
midi_stream.show('midi')
# musKAn

# Like for 200 elements the song is say for 50 sec 